In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('../data/forbes_2000_cleaned_outliers_iqr_range.csv')

In [3]:
companies = sorted(df.company.unique())
years = sorted(df.year.unique())

numeric_columns = [
    'profits',
    'assets',
    'sales',
    'cost'
]

In [4]:
# Register if company was on previous year ranking
df['was_on_last_ranking'] = False

for company in companies:
    for year in years:
        row = df.loc[(df.company == company) & (df.year == year)]
        previous = df.loc[(df.company == company) & (df.year == year - 1)]
        if not row.empty and not previous.empty:
            df.loc[(df.company == company) & (df.year == year), 'was_on_last_ranking'] = True

In [5]:
# Add yearly mean, median, std and lag_1 for numeric columns
for col in numeric_columns:
    df[f'mean_{col}'] = 0
    df[f'median_{col}'] = 0
    df[f'std_{col}'] = 0
    df[f'lag_{col}'] = 0

In [6]:
# Calculate yearly mean, median and std for numeric columns
for year in years:
    for col in numeric_columns:
        year_data = df.loc[df.year == year,col]
        df.loc[df.year == year, f'mean_{col}'] = year_data.mean()
        df.loc[df.year == year, f'median_{col}'] = year_data.median()
        df.loc[df.year == year, f'std_{col}'] = year_data.std()

In [7]:
# Calculate lag_1 feature for numeric columns
# if was_on_last_ranking == True for that company yearly ranking
for company in companies:
    for year in years:
        row = df.loc[(df.company == company) & (df.year == year) & (df.was_on_last_ranking == True)]
        if not row.empty:
            previous = df.loc[(df.company == company) & (df.year == year - 1)]
            for col in numeric_columns:
                current_value = row[col]
                previous_value = previous[col]
                lag = current_value.subtract(previous_value, fill_value=0)
                df.loc[(df.company == company) & (df.year == year), f'lag_{col}'] =  lag


In [8]:
# Describe new features
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
market_value,15496.0,11.580005,9.484464,0.000000,4.767750,8.887000,15.431000,48.059000
profits,15496.0,1.491233,2.208284,-5.495500,0.360000,0.685900,1.581250,10.560000
assets,15496.0,18.378448,19.156937,-7.380000,4.287000,12.308500,26.324000,90.230000
sales,15496.0,10.848811,9.563720,-4.390000,3.900000,8.176500,14.444000,54.600000
cost,15496.0,9.357578,9.420618,-8.200000,2.490750,6.675400,13.018325,45.719900
rank,15496.0,1181.347767,477.647479,86.000000,800.750000,1197.500000,1583.000000,2000.000000
year,15496.0,2014.692759,3.641805,2008.000000,2012.000000,2015.000000,2018.000000,2020.000000
mean_profits,15496.0,1.491233,1.634838,0.581150,0.632088,0.694715,0.768044,4.972591
median_profits,15496.0,1.301468,1.549445,0.450000,0.500000,0.529000,0.631650,4.700000
std_profits,15496.0,1.265474,0.777702,0.829251,0.839930,0.889878,1.021526,2.876189


In [9]:
industry_dummies = pd.get_dummies(df.industry, prefix='industry')
sector_dummies = pd.get_dummies(df.sector, prefix='sector')
continent_dummies = pd.get_dummies(df.continent, prefix='continent')
country_dummies = pd.get_dummies(df.country, prefix='country')

df = pd.concat(
    [df,industry_dummies,sector_dummies,continent_dummies,country_dummies],
    axis=1
)

In [10]:
df['test'] = df.year == 2020

In [11]:
df.to_csv('../data/forbes_2000_iqr_range_feature_engineering.csv', index=False, header=True)

